# Hacer

- Separar día/mes de fecha
- Codificar variables categóricas

### Definiciones

- Encoding de etiquetas: Día de la semana, se puede ordenar
- Binary encoding: Resto

## Usar

- Regresión lineal
- Árbol de decisión

https://chatgpt.com/share/671b97fb-7904-8006-853c-7d28072ea3c5

- Algún método no supervisado?

In [2]:
# Se importan librerías
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error # Se deprecó mean_squared_error y se cambió por root_mean_squared_error
import category_encoders as ce

In [3]:
# Carga del dataset
dataset = pd.read_csv("clean_data.csv")

# Se eliminan las filas que contienen "Otros" en la columna "tipo_cobro"
# Se toma esta decisión porque su impacto es mínimo
dataset = dataset[dataset["tipo_cobro"] != "Otros"]

# Se separa el día y el mes del año en dos columnas distintas
# Necesitamos que las variables sean numéricas
dataset["nro_dia"] = pd.to_datetime(dataset["fecha"]).dt.day
dataset["nro_mes"] = pd.to_datetime(dataset["fecha"]).dt.month
dataset["año"] = pd.to_datetime(dataset["fecha"]).dt.year
dataset.drop("fecha", axis=1, inplace=True)

# Se codifica la categoría día de la semana con label encoding (codificación de etiquetas)
# Se toma esta decisión porque la variable es ordinal
day_encoding = {
    'Lunes': 0,
    'Martes': 1,
    'Miercoles': 2,
    'Jueves': 3,
    'Viernes': 4,
    'Sabado': 5,
    'Domingo': 6
}

dataset['dia_semana'] = dataset['dia'].map(day_encoding)
dataset.drop("dia", axis=1, inplace=True)

# Se codifica el resto de las variables categóricas con binary encoding (codificación binaria)
binary_encoder = ce.BinaryEncoder(cols=["id_peaje", "sentido", "tipo_vehiculo", "tipo_cobro"])
dataset = binary_encoder.fit_transform(dataset)

In [4]:
# Se muestra un fragmento del dataset resultante
dataset.sample(10)

,hora,id_peaje_0,id_peaje_1,id_peaje_2,sentido_0,sentido_1,tipo_vehiculo_0,tipo_vehiculo_1,tipo_cobro_0,tipo_cobro_1,tipo_cobro_2,tipo_cobro_3,pasos,nro_dia,nro_mes,año,dia_semana
2984375,13.0,1,1,0,1,0,0,1,0,1,0,1,4,6,5,2021,3
973159,14.0,1,0,0,1,0,0,1,0,1,1,1,73,15,6,2017,3
2008977,14.0,0,0,1,0,1,0,1,0,0,1,0,105,9,1,2019,2
2281441,21.0,0,0,1,1,0,1,0,0,1,1,1,1,18,5,2019,5
3360012,13.0,1,0,0,1,0,0,1,0,1,1,0,9,17,12,2021,4
3012596,18.0,1,0,0,1,0,1,0,0,1,1,1,17,24,5,2021,0
163623,16.0,0,1,0,0,1,0,1,0,1,0,0,996,29,3,2016,1
98449,13.0,1,0,0,0,1,1,0,0,1,0,0,211,23,2,2016,1
99927,7.0,0,1,0,1,0,0,1,0,1,1,0,7,23,2,2016,1
385103,20.0,1,0,0,1,0,0,1,0,1,1,1,38,26,7,2016,1


In [5]:
# Eliminamos las columnas sentido_1 y tipo_vehiculo_1 porque no aportan información
dataset.drop(["sentido_1", "tipo_vehiculo_1"], axis=1, inplace=True)

In [6]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4473491 entries, 0 to 4473490
Data columns (total 15 columns):
 #   Column           Dtype  
---  ------           -----  
 0   hora             float64
 1   id_peaje_0       int64  
 2   id_peaje_1       int64  
 3   id_peaje_2       int64  
 4   sentido_0        int64  
 5   tipo_vehiculo_0  int64  
 6   tipo_cobro_0     int64  
 7   tipo_cobro_1     int64  
 8   tipo_cobro_2     int64  
 9   tipo_cobro_3     int64  
 10  pasos            int64  
 11  nro_dia          int32  
 12  nro_mes          int32  
 13  año              int32  
 14  dia_semana       int64  
dtypes: float64(1), int32(3), int64(11)
memory usage: 460.8 MB


In [7]:
# Cambiarle el nombre a las columnas sentido y tipo_vehículo para saber qué significa cada valor
dataset.rename(columns={'sentido_0': 'hacia_provincia'}, inplace=True)
dataset.rename(columns={'tipo_vehiculo_0': 'es_pesado'}, inplace=True)

# Regresión lineal

In [10]:
# Dividir en variables predictoras (X) y variable objetivo (y)
x = dataset[['hora', 'id_peaje_0', 'id_peaje_1', 'id_peaje_1', 'id_peaje_2', 'hacia_provincia', 'es_pesado', 'tipo_cobro_0', 'tipo_cobro_1', 'tipo_cobro_2', 'tipo_cobro_3', 'nro_dia', 'nro_mes', 'dia_semana']]  
y = dataset['pasos'] 

# Dividir en conjunto de entrenamiento y de prueba
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Instanciar el modelo
linear_model = LinearRegression()

# Entrenar el modelo
linear_model.fit(x_train, y_train)

# Predecir sobre el conjunto de prueba
y_pred = linear_model.predict(x_test)

# Evaluar el modelo
mse = root_mean_squared_error(y_test, y_pred) ** 2
print("Error cuadrático medio de la regresión lineal múltiple:", mse)


Error cuadrático medio de la regresión lineal múltiple: 182741.53143108092
